
Q1. Set up an environment for integration testing and write test cases that validate the interaction between different components of a system?

Ans: Setting up Integration Tests means validating how different parts of your application (like APIs, database, middleware, etc.) work together. Here’s a complete example using Node.js, Express, MongoDB, and Jest with Supertest to show how to set up and write integration test cases.

--supertest: Sends requests to your Express app without needing to start the server.
--mongodb-memory-server: Spins up a temporary in-memory MongoDB instance for fast, isolated tests.
--beforeAll/afterAll: Setup and teardown of MongoDB connection.
--afterEach: Cleans the DB between test cases to avoid datapollution.

1.Environment Setup
Install necessary packages:
npm install --save-dev jest supertest mongodb-memory-server

In [ ]:
##2. Sample App Code

#models/Todo.js

const mongoose = require('mongoose');

const TodoSchema = new mongoose.Schema({
  title: String,
  completed: Boolean
});

module.exports = mongoose.model('Todo', TodoSchema);



# routes/todo.js

const express = require('express');
const Todo = require('../models/Todo');
const router = express.Router();

router.get('/api/get-todo', async (req, res) => {
  const todos = await Todo.find();
  res.json(todos);
});

router.post('/api/add-todo', async (req, res) => {
  const newTodo = await Todo.create(req.body);
  res.status(200).json(newTodo);
});

module.exports = router;



# app.js

const express = require('express');
const mongoose = require('mongoose');
const todoRoutes = require('./routes/todo');

const app = express();
app.use(express.json());
app.use(todoRoutes);

module.exports = app;


In [ ]:
##3. Integration Test Example
# __tests__/todoApi.test.js

const request = require('supertest');
const mongoose = require('mongoose');
const { MongoMemoryServer } = require('mongodb-memory-server');
const app = require('../app');
const Todo = require('../models/Todo');

let mongoServer;

beforeAll(async () => {
  mongoServer = await MongoMemoryServer.create();
  const uri = mongoServer.getUri();
  await mongoose.connect(uri);
});

afterAll(async () => {
  await mongoose.disconnect();
  await mongoServer.stop();
});

afterEach(async () => {
  await Todo.deleteMany(); // Clean DB after each test
});

describe("Integration Tests for Todo API", () => {

  it("POST /api/add-todo should create a todo", async () => {
    const response = await request(app)
      .post('/api/add-todo')
      .post('/api/add-todo')
      .send({ title: 'Test Todo', completed: false });

    expect(response.statusCode).toBe(201);
    expect(response.body.title).toBe('Test Todo');
  });
  it("GET /api/get-todo should return all todos", async () => {
    await Todo.create([
      { title: 'Todo 1', completed: false },
      { title: 'Todo 2', completed: true }
    ]);

    const response = await request(app).get('/api/get-todo');
    expect(response.statusCode).toBe(200);
    expect(response.body.length).toBe(2);
  });

});



In [ ]:
###4. Run the Tests
Update package.json:

"scripts": {
  "test": "jest"
}
To Run test file use this command:

npm test